In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.engine import data_adapter
from tensorflow.keras import backend as K
import os, sys

# TOP1 InceptionV3 RSNA challenge 2017

In [19]:
from tensorflow.python.keras.engine import data_adapter
class TopModel(tf.keras.Model):  
    def __init__(self, gender=True):
        super(TopModel, self).__init__()
        
        model = tf.keras.applications.InceptionV3(include_top=False,
                            weights=None,
                            input_shape=(500,500,1),
                            pooling=None,
                            )
        gender_input = tf.keras.Input(shape=(1))
        gender_output = tf.keras.layers.Dense(32)(gender_input)
        
        model_output = tf.keras.layers.AveragePooling2D((2,2))(model.output)
        model_output = tf.keras.layers.Flatten()(model_output)
        concat_output = tf.keras.layers.Concatenate(name='gender_and_feat._concat')([model_output, gender_output])

        dense_output = tf.keras.layers.Dense(1000,activation='relu')(concat_output)
        dense_output = tf.keras.layers.Dense(1000,activation='relu')(dense_output)
        overall_output = tf.keras.layers.Dense(1,activation='relu')(dense_output)

        self.model = tf.keras.Model(inputs=[model.input, gender_input], outputs = overall_output)
            
        
        
    def call(self, x, training=False):
        imgs, labels = x
        
        y = self.model([imgs, labels],training)
       
        return y

    def train_step(self, data):
        
        data = data_adapter.expand_1d(data)
        imgs, labels, ages = data
        
        with tf.GradientTape() as tape:
            y_pred = self((imgs, labels), training=True)  # Forward pass

            loss = self.compiled_loss(ages, y_pred)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(ages, y_pred)
        _dict = {m.name: m.result() for m in self.metrics}
        
        return _dict

    def test_step(self, data):
        
        data = data_adapter.expand_1d(data)
        imgs, labels, ages = data

        y_pred = self((imgs, labels), training=False)  # Forward pass

        loss = self.compiled_loss(ages, y_pred)

        self.compiled_metrics.update_state(ages, y_pred)
        _dict = {m.name: m.result() for m in self.metrics}
        
        return _dict
    
    def summary(self):
        print(self.model.summary())

In [20]:
test = TopModel()

In [21]:
test.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 249, 249, 32) 288         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 249, 249, 32) 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 249, 249, 32) 0           batch_normalization_470[0][0]    
____________________________________________________________________________________________